# Feature Selection and Model Building

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir("/content/drive/My Drive/Capstone_Collab/")

In [4]:
ls

catboost_info/        EDA/               in_data/
Eatery_Starter.ipynb  FeatureSelection/  out_data/


In [0]:
import pandas as pd
rt = pd.read_csv('./in_data/Eatery_TrainingData.csv')

In [0]:
df=rt.copy()

In [0]:
import seaborn as sns
import numpy as np
from scipy import stats 

In [8]:
#skewness
print("Skewness: %f" % df['user_rating__aggregate_rating'].skew())

Skewness: -1.384297


### Removing rows which has zero entries more than 40% of total entries
### Replacing zeros with mean group by city id


In [0]:
df_mean=df.groupby("location__city_id").mean()


In [0]:
#Getting percentage of zeros

df_zeros=df.groupby("location__city_id")['user_rating__aggregate_rating'].apply(lambda x: (x.value_counts().get(0,0))/x.count()*100)

###Selecting records with less than 40% of zeros

In [0]:
df_zero=df_zeros[df_zeros.values<=40]

In [0]:
ind=df_zero.index

In [0]:
df=df[df["location__city_id"].isin (ind)] 

## Replacing zeros with mean grouped by city_id

In [0]:
df['user_rating__aggregate_rating'] = df.groupby("location__city_id")['user_rating__aggregate_rating'].apply(lambda x: x.replace(0,x.mean()))
df['average_cost_for_two'] = df.groupby("location__city_id")['average_cost_for_two'].apply(lambda x: x.replace(0,x.mean()))

### By doing all these Skewness decreased from -1.4 to -0.3 

In [15]:
#skewness
print("Skewness: %f" % df['user_rating__aggregate_rating'].skew())

Skewness: -0.315412


### Defining functions for creating price bins and Number of cuisines

In [0]:
def cuisine_num(df):
  
  df['cuisines'] = df['cuisines'].astype(str)
  df['num_cuisines'] = df['cuisines'].apply(lambda x: len(x.split(',')))
def price_bin(df):
  df['price_bin'] = pd.cut(df['average_cost_for_two'], bins = [0, 200, 500, 1000, 3000, 5000,10000, 800000000], labels = ['<=200', '<=500', '<=1000', '<=3000', '<=5000', '<=10000', 'no limit'])



### Converting currency notation to legible form

In [0]:
  df = df.replace(['$'], 'USD') # USA Dollar
  df = df.replace(['R$'], 'BRL') # Brazilian Real
  df = df.replace(['R'], 'SAR') # South African Rand
  df = df.replace(['NZ$'], 'NZD') # New Zealand 
  df = df.replace(['P'], 'PHP') # Philippine 
  df = df.replace(['â‚¬', '€'], 'EUR') # Portugal Euro 
  df = df.replace(['Rs.'], 'INR') # Indian Rupees 
  df = df.replace(['Â£', '£'], 'GBP') # GBP
  df = df.replace(['KÄ', 'Kč'], 'CZK') # Czech Republic
  df = df.replace(['TL'], 'TRY') # Turkish TRY
  df = df.replace(['LKR'], 'INR') # LKR Sri lankan Rupees
  df = df.replace(['LBP'], 'INR') # LBP Lebanese pound
  df = df.replace(['QR'], 'QAR') # QR Doha currency
  df = df.replace(['AED'], 'INR') # AED United Arab Emirates dirham currency


### Defining function for currency converter to INR

In [0]:
def currency_conv(df):
  currency_index = df.columns.get_loc('currency')
  cost_index = df.columns.get_loc('average_cost_for_two')

  for i in np.arange(df['id'].count()):
    
    org_currency = df.iat[i, currency_index]
    org_currency_cost = df.iat[i, cost_index]
    if org_currency == 'NZD':
        conv_value = org_currency_cost * 45.30
    elif org_currency == 'USD':
        conv_value = org_currency_cost * 71.03
    elif org_currency == 'BRL':
        conv_value = org_currency_cost * 17.39
    elif org_currency == 'SAR':
        conv_value = org_currency_cost * 4.87
    elif org_currency == 'LKR':
        conv_value = org_currency_cost * 0.39
    elif org_currency == 'LBP':
        conv_value = org_currency_cost * 0.047
    elif org_currency == 'AED':
        conv_value = org_currency_cost * 19.34
    elif org_currency == 'QAR':
        conv_value = org_currency_cost * 19.51
    elif org_currency == 'MYR':
        conv_value = org_currency_cost * 17.04
    elif org_currency == 'GBP':
        conv_value = org_currency_cost * 88.83
    elif org_currency == 'PHP':
        conv_value = org_currency_cost * 1.38
    elif org_currency == 'EUR':
        conv_value = org_currency_cost * 79.25
    elif org_currency == 'CZK':
        conv_value = org_currency_cost * 3.04
    elif org_currency == 'TRY':
        conv_value = org_currency_cost * 12.49
    elif org_currency == 'IDR':
        conv_value = org_currency_cost * 0.0051
    else:
        conv_value = org_currency_cost
    df.iat[i, cost_index] = conv_value
  df['currency'].replace(to_replace=['LKR', 'LBR', 'AED', 'QAR', 'NZD', 'USD', 'MYR', 'GBP', 'PHP', 'EUR', 'CZK', 'TRY', 'IDR', 'BRL', 'SAR'],value='INR', inplace=True)
  print("Values post replacing all currencies with INR")
  print(df['currency'].unique())

### Defining function to convert other language rating text to english

In [0]:
def cust_rating(eatery_df):
  
  keyw = ['Excellent', 'Very Good', 'Good', 'Average', 'Poor']

  excl_df1 = eatery_df[(eatery_df['user_rating__aggregate_rating'] >= 4.5) & (eatery_df['user_rating__aggregate_rating'] <= 5.0)]
  excl_df1['user_rating__rating_text'][excl_df1[~excl_df1['user_rating__rating_text'].isin(keyw)].index.tolist()]='Excellent'

  excl_df2 = eatery_df[(eatery_df['user_rating__aggregate_rating'] >= 4.0) & (eatery_df['user_rating__aggregate_rating'] <= 4.4)]
  excl_df2['user_rating__rating_text'][excl_df2[~excl_df2['user_rating__rating_text'].isin(keyw)].index.tolist()]='Very Good'

  excl_df3 = eatery_df[(eatery_df['user_rating__aggregate_rating'] >= 3.5) & (eatery_df['user_rating__aggregate_rating'] <= 3.9)]
  excl_df3['user_rating__rating_text'][excl_df3[~excl_df3['user_rating__rating_text'].isin(keyw)].index.tolist()]='Good'

  excl_df4 = eatery_df[(eatery_df['user_rating__aggregate_rating'] >= 2.5) & (eatery_df['user_rating__aggregate_rating'] <= 3.4)]
  excl_df4['user_rating__rating_text'][excl_df4[~excl_df4['user_rating__rating_text'].isin(keyw)].index.tolist()]='Average'

  excl_df5 = eatery_df[(eatery_df['user_rating__aggregate_rating'] > 0.0) & (eatery_df['user_rating__aggregate_rating'] <= 2.4)]
  excl_df5['user_rating__rating_text'][excl_df5[~excl_df5['user_rating__rating_text'].isin(keyw)].index.tolist()]='Poor'


  # Merge each dataframe into single dataframe
  frames = [excl_df1, excl_df2, excl_df3, excl_df4, excl_df5]
  eatery_df = pd.concat(frames)
  eatery_df=eatery_df.sort_index() # Sorting index
  return eatery_df

### Calling functions

In [0]:
cuisine_num(df)

In [0]:
price_bin(df)

In [22]:
currency_conv(df)

Values post replacing all currencies with INR
['INR']


In [23]:
df=cust_rating(df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:1079: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopy

# Observations

*   Zomato data set has 39 columns (one ID, one target variable (user_rating__aggregate_rating), 37 features) and 24026 records
*   book_url, medio_Provider, user_rating__rating_tool_tip, zomato_events, user_rating__custom_rating_text,user_rating__custom_rating_text_background, offers has missing values around 98%. Columns can be deleted.
*   include_bogo_offers column has constant value "True" and can be eliminated from analyis.
*   opentable_support, switch_to_order_menu,is_book_form_web_view, has constant values 0 and can be eliminated from analysis.
*   has_online_delivery, is_delivering_now are strongly correlated variables
*   is_table_reservation_supported, has_table_booking are strongly correlated
*   is_zomato_book_res,is_table_reservation_supported and has_table_booking are correlated.
*   establishment_types__establishment_type__id and establishment_types__establishment_type__name columns are inter-related as id column represents name
*   location_zipcode is dependant on location_locality so it can be removed
*   featured_image, thumb columns are having URLs to logos. These can be deleted as this doesnt add much value to the target variable.
*   location__city column can be retained and location__address, location__locality, location__locality_verbose can be removed.

In [0]:
#Deleting columns based on above observations

df=df.drop(["book_url", "medio_provider", "user_rating__rating_tool_tip", "zomato_events", "user_rating__custom_rating_text","user_rating__custom_rating_text_background","include_bogo_offers", "offers","opentable_support", "switch_to_order_menu","is_book_form_web_view","location__zipcode","featured_image", "thumb","location__address", "location__locality", "location__locality_verbose","price_range"],axis=1)

In [0]:
df=df.drop(["mezzo_provider","establishment_types__establishment_type__id","location__city_id","location__country_id","name",'location__latitude', 'location__longitude'],axis=1)

In [0]:
#Can drop currency column as the unit is "INR" for all values after conversion
df=df.drop(["currency"],axis=1)
#Drop cuisines as we are considering Fusion Number( number of cuisines particular restaurant falls into)
df=df.drop(["cuisines"],axis=1)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21218 entries, 0 to 24025
Data columns (total 14 columns):
id                                               21218 non-null int64
average_cost_for_two                             21218 non-null float64
establishment_types__establishment_type__name    21218 non-null object
has_online_delivery                              21218 non-null int64
has_table_booking                                21218 non-null int64
is_delivering_now                                21218 non-null int64
is_table_reservation_supported                   21218 non-null int64
is_zomato_book_res                               21196 non-null float64
location__city                                   21218 non-null object
user_rating__aggregate_rating                    21218 non-null float64
user_rating__rating_text                         21218 non-null object
user_rating__votes                               21218 non-null int64
num_cuisines                              

### From the above info is_zomato_book_res column has still missing values. As is_zomato_book_res and has_table_booking are correlated imputed values from the has_table_booking column. Because imputing with mean might create fractional values which is not meaningful for is_zomato_book_res

In [28]:
df['is_zomato_book_res'][df[df['is_zomato_book_res'].isnull()].index.tolist()]=df['has_table_booking'][df[df['is_zomato_book_res'].isnull()].index.tolist()]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21218 entries, 0 to 24025
Data columns (total 14 columns):
id                                               21218 non-null int64
average_cost_for_two                             21218 non-null float64
establishment_types__establishment_type__name    21218 non-null object
has_online_delivery                              21218 non-null int64
has_table_booking                                21218 non-null int64
is_delivering_now                                21218 non-null int64
is_table_reservation_supported                   21218 non-null int64
is_zomato_book_res                               21218 non-null float64
location__city                                   21218 non-null object
user_rating__aggregate_rating                    21218 non-null float64
user_rating__rating_text                         21218 non-null object
user_rating__votes                               21218 non-null int64
num_cuisines                              

In [0]:
df1=df.copy()

## n-1 Dummify

In [0]:
df1 = pd.get_dummies(df1,columns=["establishment_types__establishment_type__name","location__city","num_cuisines","price_bin","user_rating__rating_text"],drop_first=True)

In [32]:
df1.head()

,id,average_cost_for_two,has_online_delivery,has_table_booking,is_delivering_now,is_table_reservation_supported,is_zomato_book_res,user_rating__aggregate_rating,user_rating__votes,establishment_types__establishment_type__name_Bar,establishment_types__establishment_type__name_Beer Garden,establishment_types__establishment_type__name_Beverage Shop,establishment_types__establishment_type__name_Bistro,establishment_types__establishment_type__name_Café,establishment_types__establishment_type__name_Casual Dining,establishment_types__establishment_type__name_Club,establishment_types__establishment_type__name_Cocktail Bar,establishment_types__establishment_type__name_Dessert Parlour,establishment_types__establishment_type__name_Dhaba,establishment_types__establishment_type__name_Fine Dining,establishment_types__establishment_type__name_Food Court,establishment_types__establishment_type__name_Food Truck,establishment_types__establishment_type__name_Kiosk,establishment_types__establishment_type__name_Lounge,establishment_types__establishment_type__name_Microbrewery,establishment_types__establishment_type__name_Pop up,establishment_types__establishment_type__name_Pub,establishment_types__establishment_type__name_Quick Bites,establishment_types__establishment_type__name_Sandwich Shop,establishment_types__establishment_type__name_Sweet Shop,establishment_types__establishment_type__name_Wine Bar,location__city_Agra,location__city_Ajman,location__city_Aley District,location__city_Ankara,location__city_Auckland,location__city_Baabda District,location__city_Batroun District,location__city_Beirut,location__city_Beirut District,...,location__city_Saida District,location__city_San Juan City,location__city_Sandton,location__city_Santiago,location__city_Sharjah,location__city_Singapore,location__city_South East England,location__city_Soweto,location__city_Surat,location__city_São Paulo,location__city_Tagaytay City,location__city_Taguig City,location__city_Tangerang,location__city_Toronto,location__city_Tripoli,location__city_Vadodara,location__city_Varanasi,location__city_Wellington,location__city_Wellington City,location__city_West Rand,location__city_Zahle District,location__city_Zgharta,location__city_İstanbul,num_cuisines_2,num_cuisines_3,num_cuisines_4,num_cuisines_5,num_cuisines_6,num_cuisines_7,num_cuisines_8,price_bin_<=500,price_bin_<=1000,price_bin_<=3000,price_bin_<=5000,price_bin_<=10000,price_bin_no limit,user_rating__rating_text_Excellent,user_rating__rating_text_Good,user_rating__rating_text_Poor,user_rating__rating_text_Very Good
0,18888,2718.00,0,0,0,0,0.0,3.7,26,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,14145,1168.80,0,0,0,0,0.0,3.6,214,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,23643,1188.75,0,0,0,0,0.0,3.7,17,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,12767,414.00,0,0,0,0,0.0,3.4,14,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,23794,1981.25,0,0,0,0,0.0,4.1,132,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [33]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21218 entries, 0 to 24025
Columns: 140 entries, id to user_rating__rating_text_Very Good
dtypes: float64(3), int64(6), uint8(131)
memory usage: 4.3 MB


### Moving target variable to last

In [0]:
cols = list(df1.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('user_rating__aggregate_rating'))
df1 = df1[cols+['user_rating__aggregate_rating']]

In [35]:
"""
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(train_ft, thresh=4.5):
  cols = train_ft.columns
  variables = np.arange(train_ft.shape[1])
  dropped=True
  while dropped:
    
    dropped=False
    c = train_ft[cols[variables]].values
    vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]

    maxloc = vif.index(max(vif))
    if max(vif) > thresh:
      print('dropping \'' + train_ft[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
      variables = np.delete(variables, maxloc)
      dropped=True
  print('Remaining variables:')
  print(train_ft.columns[variables])
  return train_ft[cols[variables]]"""

#df1_vif=calculate_vif_(df1.iloc[:,1:-1])
#df1_vif=df1_vif.assign(id=df1.iloc[:,0].values)
#cols = list(df1_vif.columns.values)
#cols.pop(cols.index('id'))
#df1_vif = df1_vif[['id']+cols]

"\nfrom statsmodels.stats.outliers_influence import variance_inflation_factor    \n\ndef calculate_vif_(train_ft, thresh=4.5):\n  cols = train_ft.columns\n  variables = np.arange(train_ft.shape[1])\n  dropped=True\n  while dropped:\n    \n    dropped=False\n    c = train_ft[cols[variables]].values\n    vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]\n\n    maxloc = vif.index(max(vif))\n    if max(vif) > thresh:\n      print('dropping '' + train_ft[cols[variables]].columns[maxloc] + '' at index: ' + str(maxloc))\n      variables = np.delete(variables, maxloc)\n      dropped=True\n  print('Remaining variables:')\n  print(train_ft.columns[variables])\n  return train_ft[cols[variables]]"

## VIF

### Creating a function where it will add column one by one and retain useful features based on p-value (0.05)

In [0]:
import statsmodels.api as sm
def forward_regression(X, y,
                       threshold_in,
                       verbose=False):
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        if not changed:
            break

    return included

In [37]:
k=forward_regression(df1.iloc[:,1:-1],df1.iloc[:,-1],0.05) #retaining useful columns

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [0]:
k+=["id",'user_rating__aggregate_rating'] #adding id and target column to the useful features

In [0]:
df1=df1.loc[:,df1.columns.isin(k)]

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train,val = train_test_split(df1, test_size=0.30, random_state=42)

### Creating Key,features and target for training data

In [0]:
train_key=train.iloc[:,0]
train_ft=train.iloc[:,1:-1]
train_target=train.iloc[:,-1]


### Creating Key,features and target for validation data

In [0]:
val_key=val.iloc[:,0]
val_ft=val.iloc[:,1:-1]
val_target=val.iloc[:,-1]

### Normalizing using MinMax Scaler

In [44]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(train_ft)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [0]:
train_ft_sc=pd.DataFrame(scaler.transform(train_ft),columns=train_ft.columns)
val_ft_sc=pd.DataFrame(scaler.transform(val_ft),columns=val_ft.columns)
train_X=train_ft_sc
val_X=val_ft_sc

In [46]:
train_X.describe()

,is_zomato_book_res,user_rating__votes,establishment_types__establishment_type__name_Bar,establishment_types__establishment_type__name_Beverage Shop,establishment_types__establishment_type__name_Café,establishment_types__establishment_type__name_Casual Dining,establishment_types__establishment_type__name_Club,establishment_types__establishment_type__name_Dessert Parlour,establishment_types__establishment_type__name_Dhaba,establishment_types__establishment_type__name_Fine Dining,establishment_types__establishment_type__name_Food Court,establishment_types__establishment_type__name_Kiosk,location__city_Ajman,location__city_Ankara,location__city_Birmingham,location__city_Christchurch,location__city_Colombo,location__city_Dehradun,location__city_Dublin,location__city_Jakarta,location__city_Johannesburg South,location__city_London,location__city_Mysore,location__city_Praha 1,location__city_Praha 10,location__city_Praha 2,location__city_Praha 3,location__city_Praha 4,location__city_Praha 5,location__city_Praha 6,location__city_Praha 7,location__city_Praha 8,location__city_Praha 9,location__city_Praha východ,location__city_Praha západ,location__city_Rio de Janeiro,location__city_Sharjah,location__city_South East England,location__city_Soweto,location__city_Taguig City,location__city_Toronto,location__city_Varanasi,location__city_West Rand,location__city_Zgharta,price_bin_no limit,user_rating__rating_text_Excellent,user_rating__rating_text_Good,user_rating__rating_text_Poor,user_rating__rating_text_Very Good
count,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.00000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000,14852.000000
mean,0.006396,0.020360,0.088002,0.030972,0.132844,0.120522,0.027538,0.088742,0.003030,0.066523,0.052989,0.033396,0.013938,0.025047,0.017237,0.019795,0.019391,0.020132,0.024845,0.033396,0.001212,0.052922,0.012726,0.013803,0.001414,0.005050,0.002963,0.002289,0.004040,0.00202,0.002222,0.003097,0.001616,0.000067,0.000404,0.021007,0.019459,0.001683,0.000539,0.004713,0.042284,0.010234,0.000808,0.000269,0.099650,0.055548,0.361635,0.017910,0.255790
std,0.079724,0.041361,0.283307,0.173248,0.339418,0.325582,0.163652,0.284381,0.054963,0.249203,0.224020,0.179675,0.117236,0.156274,0.130157,0.139301,0.137901,0.140456,0.155658,0.179675,0.034793,0.223886,0.112091,0.116676,0.037577,0.070885,0.054351,0.047793,0.063434,0.04490,0.047086,0.055568,0.040168,0.008206,0.020096,0.143413,0.138135,0.040995,0.023203,0.068493,0.201243,0.100649,0.028414,0.016409,0.299543,0.229055,0.480490,0.132629,0.436319
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.001302,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.006138,0.000000,0.000000,0.000000,0.00000

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, mean_squared_error
from math import sqrt

In [0]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lars

In [49]:
lasso=LassoCV(alphas=[1e-3,1e-2,1e-1,1,10,20,100],cv=5,max_iter=10000)
fit_lasso=lasso.fit(train_X,train_target)
fit_lasso.alpha_
y_pred=lasso.predict(val_X)
r2_score(val_target,y_pred)
sqrt(mean_squared_error(val_target,y_pred))

0.18026541801591192

### Decision Tree Regression

In [50]:
from sklearn.tree import DecisionTreeRegressor
DTree=DecisionTreeRegressor(criterion="mse",max_depth=5) 

DTree.fit(train_X,train_target)
y_pred=DTree.predict(val_X)

print("R-square:", r2_score(val_target,y_pred))
print("RMSE: " ,sqrt(mean_squared_error(val_target,y_pred)))

R-square: 0.8878635574485177
RMSE:  0.1779799805092667


### Random Forest Regression

In [51]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=100,random_state=42)
rf.fit(train_X,train_target)
y_pred=rf.predict(val_X)
print("R-square: " ,r2_score(val_target,y_pred))
print("RMSE: " ,sqrt(mean_squared_error(val_target,y_pred)))

R-square:  0.8947991616259281
RMSE:  0.17238813377767145


### KNN Regression

In [52]:
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor()
knn.fit(train_X,train_target)
y_pred=knn.predict(val_X)
print("R-square: " ,r2_score(val_target,y_pred))
print("RMSE: " ,sqrt(mean_squared_error(val_target,y_pred)))

R-square:  0.8898292896507506
RMSE:  0.17641310443694297


### SVM Regression

In [53]:
from sklearn.svm import LinearSVR
svr=LinearSVR(random_state=42)
svr.fit(train_X,train_target)
y_pred=svr.predict(val_X)
print("R-square: " ,r2_score(val_target,y_pred))
print("RMSE: " ,sqrt(mean_squared_error(val_target,y_pred)))

R-square:  0.8844891474140437
RMSE:  0.18063801924365191


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Gradient Boosting SVM

In [119]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
gbr=HistGradientBoostingRegressor(random_state=42)
model=gbr.fit(train_X,train_target)
y_pred=gbr.predict(val_X)
print("R-square: " ,r2_score(val_target,y_pred))
print("RMSE: " ,sqrt(mean_squared_error(val_target,y_pred)))

R-square:  0.9091062740096778
RMSE:  0.16023770682178987


### XG Boost

In [149]:
import xgboost as xgb
xg_b = xgb.XGBRegressor(max_depth=6,learning_rate=0.21)
model_xgb=xg_b.fit(train_X,train_target)
y_pred = xg_b.predict(val_X)
print("R-square: " ,r2_score(val_target,y_pred))
print("RMSE: " ,sqrt(mean_squared_error(val_target,y_pred)))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[18:51:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
R-square:  0.9094514709038928
RMSE:  0.1599331413765074


## Building Sequential Neural Network

In [57]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train_X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

Using TensorFlow backend.






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               6400      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 171,265
Trainable params: 171,265
Non-trainable params: 0
_________________________________________________________________


In [58]:
NN_model.fit(train_X, train_target, epochs=20, batch_size=32, validation_split = 0.3)




Train on 10396 samples, validate on 4456 samples
Epoch 1/20
10396/10396 [==============================] - 2s 200us/step - loss: 0.5605 - mean_squared_error: 0.5605 - val_loss: 0.0340 - val_mean_squared_error: 0.0340
Epoch 2/20
10396/10396 [==============================] - 1s 129us/step - loss: 0.0346 - mean_squared_error: 0.0346 - val_loss: 0.0304 - val_mean_squared_error: 0.0304
Epoch 3/20
10396/10396 [==============================] - 1s 128us/step - loss: 0.0338 - mean_squared_error: 0.0338 - val_loss: 0.0324 - val_mean_squared_error: 0.0324
Epoch 4/20
10396/10396 [==============================] - 1s 131us/step - loss: 0.0340 - mean_squared_error: 0.0340 - val_loss: 0.0317 - val_mean_squared_error: 0.0317
Epoch 5/20
10396/10396 [==============================] - 1s 128us/step - loss: 0.0343 - mean_squared_error: 0.0343 - val_loss: 0.0344 - val_mean_squared_error: 0.0344
Epoch 6/20
10396/10396 [==============================] - 1s 132us/step - loss: 0.0356 - mean_squared_error: 

In [0]:
y_pred=NN_model.predict(val_X)

In [60]:
print("R-square: " ,r2_score(val_target,y_pred))
print("RMSE: " ,sqrt(mean_squared_error(val_target,y_pred)))

R-square:  0.8891802756367132
RMSE:  0.1769319648948042


## Of all the models, XG Boost and Gradient Boosting followed by Random forest performed best

## Saving model

In [0]:
import pickle

In [0]:
filename = './out_data/finalized_model_GB_Ramani.sav'
pickle.dump(model, open(filename, 'wb'))

## Test Data

In [0]:
df_test=pd.read_csv('./in_data/Eatery_TestData.csv')

### Working with Null data

In [0]:
df_test=df_test.dropna(subset=["currency"]) # removing currency null rows as location is showing as Dummy

In [0]:
#imputing zeros with mean groub by city id
df_test['average_cost_for_two'] = df_test.groupby("location__city_id")['average_cost_for_two'].apply(lambda x: x.replace(0,x.mean()))


In [155]:
df_test['is_zomato_book_res'][df_test[df_test['is_zomato_book_res'].isnull()].index.tolist()]=df_test['has_table_booking'][df_test[df_test['is_zomato_book_res'].isnull()].index.tolist()]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_test = df_test.replace(['$'], 'USD') # USA Dollar
df_test = df_test.replace(['R$'], 'BRL') # Brazilian Real
df_test = df_test.replace(['R'], 'SAR') # South African Rand
df_test = df_test.replace(['NZ$'], 'NZD') # New Zealand 
df_test = df_test.replace(['P'], 'PHP') # Philippine 
df_test = df_test.replace(['â‚¬', '€'], 'EUR') # Portugal Euro 
df_test = df_test.replace(['Rs.'], 'INR') # Indian Rupees 
df_test = df_test.replace(['Â£', '£'], 'GBP') # GBP
df_test = df_test.replace(['KÄ', 'Kč'], 'CZK') # Czech Republic
df_test = df_test.replace(['TL'], 'TRY') # Turkish TRY
df_test = df_test.replace(['LKR'], 'INR') # LKR Sri lankan Rupees
df_test = df_test.replace(['LBP'], 'INR') # LBP Lebanese pound
df_test = df_test.replace(['QR'], 'QAR') # QR Doha currency
df_test = df_test.replace(['AED'], 'INR') # AED United Arab Emirates dirham currency

In [0]:
cuisine_num(df_test)

In [0]:
price_bin(df_test)

In [159]:
currency_conv(df_test)

Values post replacing all currencies with INR
['INR']


In [0]:

df_test=df_test.drop(["book_url", "medio_provider", "user_rating__rating_tool_tip", "zomato_events", "user_rating__custom_rating_text","user_rating__custom_rating_text_background","include_bogo_offers", "offers","opentable_support", "switch_to_order_menu","is_book_form_web_view","location__zipcode","featured_image", "thumb","location__address", "location__locality", "location__locality_verbose","price_range"],axis=1)

In [0]:
df_test=df_test.drop(["mezzo_provider","establishment_types__establishment_type__id","location__city_id","location__country_id","name",'location__latitude', 'location__longitude'],axis=1)

### we dont have any null data based on below info

In [162]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5997 entries, 0 to 6006
Data columns (total 15 columns):
id                                               5997 non-null int64
average_cost_for_two                             5997 non-null float64
cuisines                                         5997 non-null object
currency                                         5997 non-null object
establishment_types__establishment_type__name    5997 non-null object
has_online_delivery                              5997 non-null int64
has_table_booking                                5997 non-null int64
is_delivering_now                                5997 non-null int64
is_table_reservation_supported                   5997 non-null int64
is_zomato_book_res                               5997 non-null float64
location__city                                   5997 non-null object
user_rating__rating_text                         5997 non-null object
user_rating__votes                               5997 no

### n-1 Dummify

In [0]:
df_test = pd.get_dummies(df_test,columns=["establishment_types__establishment_type__name","location__city","num_cuisines","price_bin","user_rating__rating_text"],drop_first=True)

### Selecting columns based on train data columns. "k" contains columns which are retained in training data after selecting features with p<0.05

In [0]:
df_test=df_test.loc[:,df_test.columns.isin(k)]

In [0]:
# By comparing columns of train and test dataframes missing one column in test data. So adding to get equal number of columns
df_test["location__city_Praha východ"]=0

In [0]:
test_key=df_test.iloc[:,0]
test_ft=df_test.iloc[:,1:]

In [0]:
# Normalizing by using transformation
test_ft_sc=pd.DataFrame(scaler.transform(test_ft),columns=test_ft.columns)
test_X=test_ft_sc

### Loading Gradient Boosting regression model from saved model

In [172]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(test_X)
print(result)

[4.58455767 3.12964976 2.29593837 ... 4.58910993 3.14581678 2.36830291]


In [0]:
df_pred=pd.DataFrame()

In [0]:
df_pred["id"]=test_key

In [0]:
df_pred["user_rating__aggregate_rating"]=result

In [0]:
df_pred.to_csv("./out_data/Final_rating_prediction_Ramani.csv",index=False)